# Field Plotting Demo for a Helmholtz Coil

This notebook demonstrates how to use the plotting capabilities of the `em_app` package for a system of multiple coils, specifically a Helmholtz coil. Since the plotting functions are designed to work with a single coil instance, we will create a wrapper class `HelmholtzCoil` to represent the two-coil system. This class will have its own `biot_savart` method to calculate the combined field.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mtflib import mtf
from em_app.currentcoils import RingCoil, Coil
from em_app.magneticfield import Bfield

mtf.initialize_mtf(max_order=6, max_dimension=4)
mtf.set_etol(1e-12)

## 1. Define a Helmholtz Coil Wrapper Class

In [ ]:
class HelmholtzCoil:
    def __init__(self, coil1, coil2):
        self.coil1 = coil1
        self.coil2 = coil2

    def biot_savart(self, field_points, backend="python"):
        b_field1 = self.coil1.biot_savart(field_points, backend)
        b_field2 = self.coil2.biot_savart(field_points, backend)
        
        _, vectors1 = b_field1._get_numerical_data()
        _, vectors2 = b_field2._get_numerical_data()
        
        combined_vectors = vectors1 + vectors2
        
        return Bfield(field_points=field_points, b_vectors=combined_vectors)
        
    def get_max_size(self):
        # For simplicity, we'll approximate the max size based on the individual coils
        size1 = self.coil1.get_max_size()
        size2 = self.coil2.get_max_size()
        return np.maximum(size1, size2) * 1.5 # A bit larger to be safe

    def get_center_point(self):
        # The center of the Helmholtz coil is the origin
        return np.array([0.0, 0.0, 0.0])

    def plot_1d_field(self, *args, **kwargs):
        Coil.plot_1d_field(self, *args, **kwargs)

    def plot_2d_field(self, *args, **kwargs):
        Coil.plot_2d_field(self, *args, **kwargs)

    def plot_field_vectors_3d(self, *args, **kwargs):
        Coil.plot_field_vectors_3d(self, *args, **kwargs)

## 2. Define the Helmholtz Coil Configuration

In [ ]:
radius = 0.5
separation = 0.5
current = 1.0
num_segments = 20

# First coil
center1 = np.array([0, 0, -separation / 2])
axis1 = np.array([0, 0, 1])
coil1 = RingCoil(current, radius, num_segments, center1, axis1)

# Second coil
center2 = np.array([0, 0, separation / 2])
axis2 = np.array([0, 0, 1])
coil2 = RingCoil(current, radius, num_segments, center2, axis2)

helmholtz_coil = HelmholtzCoil(coil1, coil2)

## 3. Demonstrate `plot_1d_field`

In [ ]:
start = np.array([0.01, 0, -1])
end = np.array([0.01, 0, 1])
fig, ax = plt.subplots()
helmholtz_coil.plot_1d_field(field_component="Bz", start_point=start, end_point=end, ax=ax)
plt.show()

## 4. Demonstrate `plot_2d_field`

In [ ]:
center = np.array([0, 0.01, 0])
normal = np.array([0, 1, 0])
fig, ax = plt.subplots()
helmholtz_coil.plot_2d_field(
    plot_type="quiver",
    plane="xz",
    center=center,
    size_a=2,
    size_b=2,
    num_points_a=15,
    num_points_b=15,
    ax=ax
)
plt.show()

## 5. Demonstrate `plot_field_vectors_3d`

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection="3d")
helmholtz_coil.plot_field_vectors_3d(ax=ax, num_points_a=5, num_points_b=5, num_points_c=5)
plt.show()